## Introduction to `robotoc` 
# 4: Optimal Control of a Quadruped

## 本章の目的
ここでは，４脚ロボット ANYmal の最適制御問題（OCP）を定式化し，解くことが目的です．

まず，ANYmal の `robotoc.Robot` モデルを作成します．

In [ ]:
import robotoc

model_info = robotoc.RobotModelInfo()
model_info.urdf_path = 'urdf/anymal_b_simple_description/urdf/anymal.urdf'
model_info.base_joint_type = robotoc.BaseJointType.FloatingBase
baumgarte_time_step = 0.05
model_info.point_contacts = [robotoc.ContactModelInfo('LF_FOOT', baumgarte_time_step),
                             robotoc.ContactModelInfo('LH_FOOT', baumgarte_time_step),
                             robotoc.ContactModelInfo('RF_FOOT', baumgarte_time_step),
                             robotoc.ContactModelInfo('RH_FOOT', baumgarte_time_step)]
anymal = robotoc.Robot(model_info)

## Cost Function（評価関数）
次に，評価関数 (cost function) を作成します．   
`robotoc`では，様々な評価関数のコンポーネント（**cost components**）を組み合わせて評価関数, `robotoc.CostFunction` (`std::shared_ptr<robotoc::CostFunction>` in C++), を作ります．  
各 **cost component** はある特定の評価関数の項を表しており，`robotoc.CostFunctionComponentBase` (`robotoc::CostFunctionComponentBase` in C++) を継承しています.   
**cost component** を評価関数 `robotoc.CostFunction` に集め， 評価関数 `robotoc.CostFunction` をソルバーに渡します．   
このようにすることで，複数の**cost components** を簡単に組み合わせて評価関数が設計できるようになっています.

基本的な **cost components** は `robotoc` のパッケージに既に多数実装されています.  
この例では， configuration つまり関節角，角速度，トルクなどについての **cost component**, task-space つまり end-effector (振り脚の軌道)についての **cost component**，そして重心 (CoM) 位置についての **cost component** を考えます．   
以下がその **cost component** です．

### Gaite Pattern (歩行パターン)
評価関数を作る前に，まずは歩行パターンを設定します．　　   
以下のようなシンプルなトロットを考えます． 

In [ ]:
import numpy as np
step_length = np.array([0.15, 0, 0])
step_height = 0.1
swing_time = 0.5
double_support_time = 0.04
swing_start_time = 0.04

### Configuration Space Cost

レファレンスと重みパラメータを設定していきます．   
以下のような命名規則があります．   

- `q_ref`: configuration のレファレンス (目標値）
- `q_weight`: configuration の重みパラメータ
- `v_weight`: 一般化速度 (関節角速度) の重みパラメータ
- `a_weight`: 一般化加速度 (関節角加速度) の重みパラメータ
- `u_weight`: 関節トルクの重みパラメータ
- `q_weight_terminal`, `v_weight_terminal`: ホライゾン終端での configuration と一般化速度の重みパラメータ
- `q_weight_impact`, `v_weight_impact`: 衝突の瞬間での configuration と一般化速度の重みパラメータ
- `dv_weight_impact`: 衝突の瞬間での 一般化速度変化（状態ジャンプ）の重みパラメータ

In [ ]:
q_standing = np.array([0, 0, 0.4792, 0, 0, 0, 1, 
                       -0.1,  0.7, -1.0, 
                       -0.1, -0.7,  1.0, 
                        0.1,  0.7, -1.0, 
                        0.1, -0.7,  1.0])
q_weight = np.array([0, 0, 0, 250000, 250000, 250000, 
                     0.0001, 0.0001, 0.0001, 
                     0.0001, 0.0001, 0.0001,
                     0.0001, 0.0001, 0.0001,
                     0.0001, 0.0001, 0.0001])
v_weight = np.array([100, 100, 100, 100, 100, 100, 
                     1, 1, 1, 
                     1, 1, 1,
                     1, 1, 1,
                     1, 1, 1])
u_weight = np.full(anymal.dimu(), 1.0e-01)
q_weight_terminal = q_weight
v_weight_terminal = v_weight
q_weight_impact = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 
                            100, 100, 100, 
                            100, 100, 100,
                            100, 100, 100,
                            100, 100, 100])
v_weight_impact = np.full(anymal.dimv(), 100)
config_cost = robotoc.ConfigurationSpaceCost(anymal)
config_cost.set_q_ref(q_standing)
config_cost.set_q_weight(q_weight)
config_cost.set_v_weight(v_weight)
config_cost.set_u_weight(u_weight)
config_cost.set_q_weight_terminal(q_weight_terminal)
config_cost.set_v_weight_terminal(v_weight_terminal)
config_cost.set_q_weight_impact(q_weight_impact)
config_cost.set_v_weight_impact(v_weight_impact)

### Task Space Cost
次に脚先の位置追従を表す，タスク空間の cost component を，各脚に合計4つ定義していきます．   
このような cost component は， `robotoc.TaskSpace3DCost` (`std::shared_ptr<robotoc::TaskSpace3DCost>` in C++) を用いて定義できます.  
各 `robotoc.TaskSpace3DCost`では, `robotoc.TaskSpace3DRefBase` (`robotoc::TaskSpace3DRefBase` in C++) を継承したレファレンスクラスを指定する必要があります．   
シンプルな歩行に対して， `robotoc.PeriodicSwingFootRef` (`robotoc::PeriodicSwingFootRef` in C++) が既に実装されています．   
これには， initial position (初期位置), step length (ステップの距離), step height (振り足の目標高さ), swing-start time (その脚が振り始める時間), swing time (スイングしている時間), support time (地面に立っている時間), and whether the initial step-length is half or not, を指定します.

まず各脚の初期位置を得るために，直立している姿勢（デフォルトの姿勢）での各脚の位置を forward kinematics から得ます．

In [ ]:
anymal.forward_kinematics(q_standing)
x3d0_LF = anymal.frame_position('LF_FOOT')
x3d0_LH = anymal.frame_position('LH_FOOT')
x3d0_RF = anymal.frame_position('RF_FOOT')
x3d0_RH = anymal.frame_position('RH_FOOT')

これと歩行パターンに基づいて時変の脚先位置のレファレンスをそれぞれの脚について作ります．

In [ ]:
LF_t0 = swing_start_time + swing_time + double_support_time
LH_t0 = swing_start_time
RF_t0 = swing_start_time
RH_t0 = swing_start_time + swing_time + double_support_time
LF_foot_ref = robotoc.PeriodicSwingFootRef(x3d0_LF, step_length, step_height, 
                                           LF_t0, swing_time, 
                                           swing_time+2*double_support_time, False)
LH_foot_ref = robotoc.PeriodicSwingFootRef(x3d0_LH, step_length, step_height, 
                                           LH_t0, swing_time, 
                                           swing_time+2*double_support_time, True)
RF_foot_ref = robotoc.PeriodicSwingFootRef(x3d0_RF, step_length, step_height, 
                                           RF_t0, swing_time, 
                                           swing_time+2*double_support_time, True)
RH_foot_ref = robotoc.PeriodicSwingFootRef(x3d0_RH, step_length, step_height, 
                                           RH_t0, swing_time, 
                                           swing_time+2*double_support_time, False)

このレファレンスを用いて脚先位置追従の cost component を作ります．

In [ ]:
LF_cost = robotoc.TaskSpace3DCost(anymal, 'LF_FOOT', LF_foot_ref)
LH_cost = robotoc.TaskSpace3DCost(anymal, 'LH_FOOT', LH_foot_ref)
RF_cost = robotoc.TaskSpace3DCost(anymal, 'RF_FOOT', RF_foot_ref)
RH_cost = robotoc.TaskSpace3DCost(anymal, 'RH_FOOT', RH_foot_ref)
swing_foot_weight = np.full(3, 1.0e06)
LF_cost.set_weight(swing_foot_weight)
LH_cost.set_weight(swing_foot_weight)
RF_cost.set_weight(swing_foot_weight)
RH_cost.set_weight(swing_foot_weight)

### Center of Mass (CoM) Cost

脚先と同様に，重心位置 (CoM) の cost component を作ります．   
まずは，CoMの位置についての時変のレファレンスを作ります．

In [ ]:
com_ref0 = anymal.com()
vcom_ref = 0.5 * step_length / swing_time
com_ref = robotoc.PeriodicCoMRef(com_ref0, vcom_ref, swing_start_time, swing_time, 
                                 double_support_time, True)

このレファレンスから，CoM位置の cost component を作ります．

In [ ]:
com_cost = robotoc.CoMCost(anymal, com_ref)
com_cost.set_weight(np.full(3, 1.0e06))

以上の **cost components** を`robotoc.CostFunction`にまとめます．

In [ ]:
cost = robotoc.CostFunction()
cost.add("config_cost", config_cost)
cost.add("LF_cost", LF_cost)
cost.add("LH_cost", LH_cost)
cost.add("RF_cost", RF_cost)
cost.add("RH_cost", RH_cost)
cost.add("com_cost", com_cost)
print(cost)

## Constraints（制約）
制約についても評価関数と同様に作成します．   
`robotoc.Constraints` (`std::shared_ptr<robotoc::Constraints>` in C++) は **constraint components**のコレクションになっています.
各 **constraints component** はある特定の制約を表しており， `robotoc.ConstraintComponentBase` (`robotoc::ConstraintComponentBase` in C++) を継承しています．     
これにより，様々な **constraints components** を簡単に組み合わせて制約を設計できます.

基本的な **constraint components** のは既に `robotoc` に多数実装されています.  
この例では，関節角度，角速度，トルクの制約を考えます．     

In [ ]:
joint_position_lower  = robotoc.JointPositionLowerLimit(anymal)
joint_position_upper  = robotoc.JointPositionUpperLimit(anymal)
joint_velocity_lower  = robotoc.JointVelocityLowerLimit(anymal)
joint_velocity_upper  = robotoc.JointVelocityUpperLimit(anymal)
joint_torques_lower   = robotoc.JointTorquesLowerLimit(anymal)
joint_torques_upper   = robotoc.JointTorquesUpperLimit(anymal)

さらに，摩擦錐 (friction cone) 制約をつくります．    

In [ ]:
friction_cone = robotoc.FrictionCone(anymal)

制約を作り，上記のコンポーネントを加えます．

In [ ]:
constraints = robotoc.Constraints()
constraints.add("joint_position_lower", joint_position_lower)
constraints.add("joint_position_upper", joint_position_upper)
constraints.add("joint_velocity_lower", joint_velocity_lower)
constraints.add("joint_velocity_upper", joint_velocity_upper)
constraints.add("joint_torques_lower", joint_torques_lower)
constraints.add("joint_torques_upper", joint_torques_upper)
constraints.add("friction_cone", friction_cone)
print(constraints)

## Contact Sequence（接触系列）
**Contact sequence** (`robotoc.ContactSequence`) は **contact status**  (`robotoc.ContactStatus`) の系列です．   
**Contact status** は接触についての情報，どの接触がアクティブなのか？ や接触の位置など，を記述したものです.  
これをOCPソルバーに渡すことで，OCPソルバーは自動的に問題を contact sequence に合わせて定式化してくれます．

この例では，次の回数だけのトロットを考えます．

In [ ]:
cycle = 3

以下のように contact sequence を作ります．

In [ ]:
contact_sequence = robotoc.ContactSequence(anymal)

ここから，逐次的に contact sequence を作り上げていきます．   
まずは，初期状態での `robotoc.ContactStatus`をつくります．   
この`robotoc.ContactStatus`に接触状況（接触がアクティブか否か，接触の位置）を記述します．   
初期状態ではロボットは `q_standing`の姿勢をとって直立しています．　  
つまり，全ての脚で接触はアクティブです．      
従って，以下のように記述します．

In [ ]:
anymal.forward_kinematics(q_standing)
x3d0_LF = anymal.frame_position('LF_FOOT')
x3d0_LH = anymal.frame_position('LH_FOOT')
x3d0_RF = anymal.frame_position('RF_FOOT')
x3d0_RH = anymal.frame_position('RH_FOOT')
contact_positions = {'LF_FOOT': x3d0_LF, 'LH_FOOT': x3d0_LH, 'RF_FOOT': x3d0_RF, 'RH_FOOT': x3d0_RH} 
mu = 0.7
friction_coefficients = {'LF_FOOT': mu, 'LH_FOOT': mu, 'RF_FOOT': mu, 'RH_FOOT': mu} 
contact_status_standing = anymal.create_contact_status()
contact_status_standing.activate_contacts(['LF_FOOT', 'LH_FOOT', 'RF_FOOT', 'RH_FOOT'])
contact_status_standing.set_contact_placements(contact_positions)
contact_status_standing.set_friction_coefficients(friction_coefficients)

これを用いて contact sequence を初期化します．

In [ ]:
contact_sequence.init(contact_status_standing)

次に， `LF_FOOT` と `RH_FOOT` の接触がアクティブ（地面についている）で， `LH_FOOT`　と `RF_FOOT` が振り脚になっている状況を考えます． 

In [ ]:
contact_status_lhrf_swing = anymal.create_contact_status()
contact_status_lhrf_swing.activate_contacts(['LF_FOOT', 'RH_FOOT'])
contact_status_lhrf_swing.set_contact_placements(contact_positions)

こちらを contact sequence に加えます．(push_backします．)

In [ ]:
contact_sequence.push_back(contact_status_lhrf_swing, swing_start_time)

次に，再び４つの脚が地面につきます．   
振り脚であった `LH_FOOT` と　`RF_FOOT` は 0.5 x `step_length` だけ進んでいます.   
こちらは以下のように書くことができます．

In [ ]:
contact_positions['LH_FOOT'] += 0.5 * step_length
contact_positions['RF_FOOT'] += 0.5 * step_length
contact_status_standing.set_contact_placements(contact_positions)
contact_sequence.push_back(contact_status_standing, swing_start_time+swing_time)

次は， `LF_FOOT` と `RH_FOOT` が地面についていて（接触がアクティブ）， `LF_FOOT` と `RH_FOOT` が振り脚（接触がアクティブでない）状況を考えます．   

In [ ]:
contact_status_lfrh_swing = anymal.create_contact_status()
contact_status_lfrh_swing.activate_contacts(['LH_FOOT', 'RF_FOOT'])
contact_status_lfrh_swing.set_contact_placements(contact_positions)
contact_sequence.push_back(contact_status_lfrh_swing, 
                           swing_start_time+swing_time+double_support_time)

その次は，再び4脚が地面につきます．  
振り脚だった脚は `step_length` だけ進んでいます．

In [ ]:
contact_positions['LF_FOOT'] += step_length
contact_positions['RH_FOOT'] += step_length
contact_status_standing.set_contact_placements(contact_positions)
contact_sequence.push_back(contact_status_standing, 
                           swing_start_time+2*swing_time+double_support_time)

これを指定した cycle の数だけ繰返します．

In [ ]:
for i in range(cycle-1):
    t1 = swing_start_time + (i+1)*(2*swing_time+2*double_support_time)
    contact_status_lhrf_swing.set_contact_placements(contact_positions)
    contact_sequence.push_back(contact_status_lhrf_swing, t1)

    contact_positions['LH_FOOT'] += step_length
    contact_positions['RF_FOOT'] += step_length
    contact_status_standing.set_contact_placements(contact_positions)
    contact_sequence.push_back(contact_status_standing, t1+swing_time)

    contact_status_lfrh_swing.set_contact_placements(contact_positions)
    contact_sequence.push_back(contact_status_lfrh_swing, 
                               t1+swing_time+double_support_time)

    contact_positions['LF_FOOT'] += step_length
    contact_positions['RH_FOOT'] += step_length
    contact_status_standing.set_contact_placements(contact_positions)
    contact_sequence.push_back(contact_status_standing, 
                               t1+2*swing_time+double_support_time)

## Optimal Control Problem (OCP) and Solver

ここでは最適制御問題 (OCP) とそのソルバーを作ります．  
まずはホライゾン長さ `T` と離散化のグリッドの数 `N` を設定しましょう． 

In [ ]:
T = swing_start_time + cycle*(2*double_support_time+2*swing_time)
dt = 0.02
N = int(np.floor(T/dt))    

最適制御問題 (OCP), `robotoc.OCP` (`robotoc::OCP` in C++) を用いて問題を記述します.     

In [ ]:
ocp = robotoc.OCP(robot=anymal, cost=cost, constraints=constraints, 
                  contact_sequence=contact_sequence, T=T, N=N)

ではこのOCPのソルバー`robotoc.OCPSolver` (`robotoc::OCPSolver` in C++)　を作ります．   
このソルバーは以下の特徴があります: 
- Direct multiple-shooting 法　と primal-dual interior point 法 （主双対内点法）
- Inverse dynamics を用いた **unconstrained** OCP　 (i.e., no contacts or floating bases) の効率的なアルゴリズム.
- 状態のみの等式制約のための制約変換法
- Riccati recursion によるニュートンステップの計算
- スイッチ時刻最適化可能な Riccati recursion アルゴリズム（この例では用いていません）

種々のソルバーのオプションを `robotoc.SolverOptions` により設定できます．(ここではデフォルトのパラメータを用います).  
例えば，ソルバー作成時に並列計算で用いるスレッドの数を`nthreads`で指定します.

In [ ]:
solver_options = robotoc.SolverOptions()
solver_options.nthreads = 4
ocp_solver = robotoc.OCPSolver(ocp=ocp, solver_options=solver_options)

## Solve the OCP

初期状態 (`q`, `v`) と初期時刻 (`t`) が以下で与えられるOCP（最適制御問題）を考えます．

In [ ]:
t = 0.
q = q_standing.copy()
v = np.zeros(anymal.dimv())

まずはソルバーを初期化します．   
以下は初期推定解を上の`q`, `v`に設定します．    
さらに，接触力`f`の推定解をロボットの weight から設定します．   
これは非常にシンプルですが高速な収束に割と重要です．   
(例えば状態方程式を用いるといったような，より良い初期化の方法もあると思います．しかし適当な初期化でも direct multiple-shooting の収束性能は良く実装は面倒なため実装していません．)

In [ ]:
ocp_solver.discretize(t)
ocp_solver.set_solution("q", q)
ocp_solver.set_solution("v", v)
f_init = np.array([0.0, 0.0, 0.25*anymal.total_weight()])
ocp_solver.set_solution("f", f_init)

また，不等式制約に関してソルバーを初期化します．    
(具体的には，主双対内点法のスラック変数とラグランジュ乗数を初期化しますが，特に内部について意識する必要はありません).

In [ ]:
ocp_solver.init_constraints()

ソルバーの現在の推定解がどの程度最適解に近いか，ということを判断するために，本ソルバーでは **KKT error** という指標を用います．   
これは， Karush–Kuhn–Tucker (KKT) 条件（1次の最適性の必要条件）のl2ノルムです．(最適解でこれは0になります)      
まず，初期化後の KKT error を見てみます． 

In [ ]:
ocp_solver.KKT_error(t, q, v)

次にOCPを解きます．

In [ ]:
ocp_solver.solve(t, q, v)

KKT error がとても小さくなっていることが分かります．    
(具体的には，`robotoc.SolverOption`で設定した閾値より小さくなっています).

In [ ]:
ocp_solver.KKT_error(t, q, v)

また，各反復での KKT error やステップサイズといった情報も以下で見れます．

In [ ]:
stats = ocp_solver.get_solver_statistics()
print(stats)

最適軌道のconfiguration, 一般化速度，関節トルクをプロットしてみます．   

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn

plt.rcParams['figure.figsize'] = 10, 9
fig = plt.figure()
ax_q = fig.add_subplot(3, 1, 1)
ax_v = fig.add_subplot(3, 1, 2)
ax_u = fig.add_subplot(3, 1, 3)

# Plot results
time_discretization = ocp_solver.get_time_discretization()
t = [time_discretization[i].t for i in range(len(time_discretization))]
ax_q.plot(t, ocp_solver.get_solution('q'))
ax_v.plot(t, ocp_solver.get_solution('v'))
ax_u.plot(t, ocp_solver.get_solution('u'))

ax_q.set_xlabel('')
ax_v.set_xlabel('')
ax_u.set_xlabel('t [s]')

ax_q.set_ylabel('q [rad]')
ax_v.set_ylabel('v [rad/s]')
ax_u.set_ylabel('u [Nm]')

`robotoc` は接触力を friction cone 制約とともにプロットするモジュールも提供しています．   
グレーのハッチが fx と fy の infeasible な領域を表しています.

In [ ]:
plot_f = robotoc.utils.PlotContactForce(mu=mu)
plot_f.figsize = 14, 8
plot_f.wspace = 0.2
plot_f.hspace = 0.2
plot_f.legend_bbox_to_anchor = (-0.2, 2.7)
plot_f.plot(f_traj=ocp_solver.get_solution('f', 'WORLD'), 
            time_discretization=ocp_solver.get_time_discretization())

また収束をプロットするモジュールもあります．

In [ ]:
plot_kkt = robotoc.utils.PlotConvergence()
#plot_kkt.ylim = [0., 1.5]
plot_kkt.figsize = 6, 4
kkt_data = [e.kkt_error for e in ocp_solver.get_solver_statistics().performance_index]
plot_kkt.plot(kkt_data=kkt_data)

また，以下で最適化された軌道を可視化できます．   
`'gepetto'` を `viewer_type` に選べば， 接触力および friction cone 制約も可視化できます．

In [ ]:
viewer = robotoc.utils.TrajectoryViewer(model_info=model_info,
                                        viewer_type='gepetto')
viewer.set_contact_info(mu)
viewer.display(ocp_solver.get_time_discretization(), 
               ocp_solver.get_solution('q'), 
               ocp_solver.get_solution('f', 'WORLD'))